In [1]:
from pyspark import SparkContext,SparkConf
conf = SparkConf().setMaster("local").setAppName("Temperature")
sc = SparkContext(conf = conf)

In [2]:
lines = sc.textFile("/home/jovyan/work/temperature/1800.csv")
lines.take(10)

['ITE00100554,18000101,TMAX,-75,,,E,',
 'ITE00100554,18000101,TMIN,-148,,,E,',
 'GM000010962,18000101,PRCP,0,,,E,',
 'EZE00100082,18000101,TMAX,-86,,,E,',
 'EZE00100082,18000101,TMIN,-135,,,E,',
 'ITE00100554,18000102,TMAX,-60,,I,E,',
 'ITE00100554,18000102,TMIN,-125,,,E,',
 'GM000010962,18000102,PRCP,0,,,E,',
 'EZE00100082,18000102,TMAX,-44,,,E,',
 'EZE00100082,18000102,TMIN,-130,,,E,']

In [3]:
def parseLine(line): 
    fields = line.split(",")
    stationId = fields[0]
    temperatureType = fields[2]
    temperature = float(fields[3]) * 0.1 * (9.0 / 5.0) + 32.0
    return (stationId, temperatureType, temperature)

In [5]:
parsedLines = lines.map(parseLine)
#parsedLines = lines.map(lambda x: parseLine(x))
parsedLines.take(10)
#parsedLines_bis = lines.map(lambda x : (x.split(",")[0], x.split(",")[2], float(x.split(",")[3]) * 0.1 * (9.0 / 5.0) + 32.0))
#parsedLines_bis.take(10)              

[('ITE00100554', 'TMAX', 18.5),
 ('ITE00100554', 'TMIN', 5.359999999999999),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', 16.52),
 ('EZE00100082', 'TMIN', 7.699999999999999),
 ('ITE00100554', 'TMAX', 21.2),
 ('ITE00100554', 'TMIN', 9.5),
 ('GM000010962', 'PRCP', 32.0),
 ('EZE00100082', 'TMAX', 24.08),
 ('EZE00100082', 'TMIN', 8.599999999999998)]

In [6]:
minTemps = parsedLines.filter(lambda x : x[1]=='TMIN')
minTemps = parsedLines.filter(lambda x: "TMIN" in x[1])
minTemps.take(10)

[('ITE00100554', 'TMIN', 5.359999999999999),
 ('EZE00100082', 'TMIN', 7.699999999999999),
 ('ITE00100554', 'TMIN', 9.5),
 ('EZE00100082', 'TMIN', 8.599999999999998),
 ('ITE00100554', 'TMIN', 23.72),
 ('EZE00100082', 'TMIN', 18.86),
 ('ITE00100554', 'TMIN', 29.66),
 ('EZE00100082', 'TMIN', 18.68),
 ('ITE00100554', 'TMIN', 30.919999999999998),
 ('EZE00100082', 'TMIN', 21.56)]

In [7]:
#Nombre de stations :
stationNumber = minTemps.map(lambda x: x[0])
stationNumber.take(3)
stationNumber = stationNumber.distinct()
stationNumber.count()

2

In [8]:
minTemps = minTemps.map(lambda x : (x[0],x[2]))

In [11]:
minTemps.take(10)

[('ITE00100554', 5.359999999999999),
 ('EZE00100082', 7.699999999999999),
 ('ITE00100554', 9.5),
 ('EZE00100082', 8.599999999999998),
 ('ITE00100554', 23.72),
 ('EZE00100082', 18.86),
 ('ITE00100554', 29.66),
 ('EZE00100082', 18.68),
 ('ITE00100554', 30.919999999999998),
 ('EZE00100082', 21.56)]

In [9]:
minTempsByStation = minTemps.reduceByKey(lambda x,y : min(x,y))

In [10]:
minTempsByStation.take(10)

[('ITE00100554', 5.359999999999999), ('EZE00100082', 7.699999999999999)]